# Deep Learning Toolkit for Splunk - Online Learning with Half Space Trees


This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [2]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
from river import anomaly
from river import compose
from river import datasets
from river import metrics
from river import preprocessing
import pickle
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [3]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
import river
print("river version: " + river.__version__)

numpy version: 1.21.4
pandas version: 1.3.4
river version: 0.8.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [5]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [6]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("online_anomaly_detection")
print(df.describe())
print(df.head())
print(param)

        Recruiting
count    91.000000
mean    229.890110
std     244.979113
min       7.000000
25%      42.500000
50%     247.000000
75%     305.500000
max    2168.000000
   Recruiting
0          33
1          30
2         297
3         308
4         305
{'options': {'params': {'mode': 'stage', 'algo': 'river_halfspacetree'}, 'args': ['Recruiting'], 'feature_variables': ['Recruiting'], 'model_name': 'online_anomaly_detection', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'streaming_apply': 'false', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['Recruiting']}


## Stage 2 - create and initialize a model

https://riverml.xyz/dev/api/anomaly/HalfSpaceTrees/

Parameters
- n_trees – defaults to 10
Number of trees to use.

- height – defaults to 8
Height of each tree. Note that a tree of height h is made up of h + 1 levels and therefore contains 2 ** (h + 1) - 1 nodes.

- window_size – defaults to 250
Number of observations to use for calculating the mass at each node in each tree.

- limits (Dict[Hashable, Tuple[float, float]]) – defaults to None
Specifies the range of each feature. By default each feature is assumed to be in range [0, 1].

- seed (int) – defaults to None
Random number seed.

In [12]:
# Create the random cut forest from the source data
def init(df,param):
    # Set model parameters
    X = df[param['feature_variables'][0]]    
    n_trees=10
    height=8
    window_size=250
    if 'options' in param:
        if 'params' in param['options']:
            if 'n_trees' in param['options']['params']:
                n_trees = int(param['options']['params']['n_trees'])
            if 'height' in param['options']['params']:
                height = int(param['options']['params']['height'])
            if 'window_size' in param['options']['params']:
                window_size = int(param['options']['params']['window_size'])
    
    # Create the half space tree
    model = compose.Pipeline(
        preprocessing.MinMaxScaler(),
        anomaly.HalfSpaceTrees(
            n_trees=n_trees,
            height=height,
            window_size=window_size,
            seed=42)
    )

    return model

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model=init(df,param)

In [14]:
model

Pipeline (
  MinMaxScaler (),
  HalfSpaceTrees (
    n_trees=10
    height=8
    window_size=250
    limits=defaultdict(..., {})
    seed=42
  )
)

## Stage 3 - fit the model

In [15]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    X = df[param['feature_variables'][0]]
    # init with a few warm up samples
    for x in X[:10]:
        model = model.learn_one({'x': x})
    return len(X)

In [16]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

91


## Stage 4 - apply the model

In [22]:
# apply your model
# returns the calculated results
def apply(model,df,param):

    X = df[param['feature_variables'][0]]
    Y = []
    
    for x in X:
        features = {'x': x}
        model = model.learn_one(features)
        score = model.score_one(features)
        Y.append(score)        
        #print(f'Anomaly score for x={x:.3f}: {model.score_one(features):.3f}')

    # save the model
    if 'options' in param:
        if 'model_name' in param['options']:
            if 'params' in param['options']:
                if 'algo' in param['options']['params']:
                    name = param['options']['params']['algo'] + '_' + param['options']['model_name']
                    save(model,name)
                    #print('/apply updated and saved model with parameters ', model)
                    
    
    result=pd.DataFrame({'anomaly_score':Y})
    return result

In [30]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
results=apply(model,df,param)
results

,anomaly_score
0,0.814780
1,0.817586
2,0.711616
3,0.706582
4,0.707032
...,...
86,0.698981
87,0.709301
88,0.709627
89,0.987604


## Stage 5 - save the model

In [19]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + '.pkl', 'wb') as f:
        pickle.dump(model, f)
    return model

## Stage 6 - load the model

In [20]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + '.pkl', 'rb') as f:
        model = pickle.load(f)
    return model

## Stage 7 - provide a summary of the model

In [21]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code